In [5]:
# Notebook-Zelle 1: Imports & Graph laden
from rdflib import Graph, Namespace, RDF, URIRef
from pathlib import Path

# Namespace definieren
AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")

# Pfad zur TTL-Datei (angepasst nach deinem Speicherort)
ttl_path = Path(r"D:\MA_Python_Agent\ParamDiag_Agent.ttl")

# Graph laden
g = Graph()
g.parse(ttl_path.as_posix(), format="turtle")
print("Initiale Tripelanzahl:", len(g))



Initiale Tripelanzahl: 866


In [6]:
# Notebook-Zelle 2: Löschroutine definieren
def delete_instances_of_class(graph, cls):
    """
    Löscht alle Tripel, bei denen eine Ressource den Typ `cls` hat,
    sowie alle Tripel, bei denen diese Ressource als Subjekt auftritt.
    """
    # finde alle Instanzen der Klasse
    instances = list(graph.subjects(RDF.type, cls))
    print(f"Gefundene Instanzen von {cls}: {len(instances)}")
    # für jede Instanz: alle Tripel mit dieser Instanz als Subjekt entfernen
    for inst in instances:
        graph.remove((inst, None, None))
    # ggf. auch Tripel, bei denen diese Instanz als Objekt verwendet wird, entfernen:
    for inst in instances:
        graph.remove((None, None, inst))
    return len(instances)

# Notebook-Zelle 3: Instanzen löschen
# Lösche alle Instanzen der Klasse class_Program
num_prog = delete_instances_of_class(g, AG.class_Program)
print(f"Instanzen class_Program gelöscht: {num_prog}")
print("Tripel nach Löschen class_Program:", len(g))

# Lösche alle Instanzen der Klasse class_Variable
num_var = delete_instances_of_class(g, AG.class_Variable)
print(f"Instanzen class_Variable gelöscht: {num_var}")
print("Tripel nach Löschen class_Variable:", len(g))

outpath = Path(r"D:\MA_Python_Agent\ParamDiag_Agent.ttl")
g.serialize(destination=outpath.as_posix(), format="turtle")
print("Bereinigter Graph unter:", outpath)
print("Finale Tripelanzahl:", len(g))




Gefundene Instanzen von http://www.semanticweb.org/AgentProgramParams/class_Program: 50
Instanzen class_Program gelöscht: 50
Tripel nach Löschen class_Program: 260
Gefundene Instanzen von http://www.semanticweb.org/AgentProgramParams/class_Variable: 260
Instanzen class_Variable gelöscht: 260
Tripel nach Löschen class_Variable: 0
Bereinigter Graph unter: D:\MA_Python_Agent\ParamDiag_Agent.ttl
Finale Tripelanzahl: 0
